In [1]:
import numpy as np
import pandas as pd
import os

# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# from scipy.special import rel_entr
# from scipy.stats import entropy
# import scipy.stats
import math
import warnings 
warnings.filterwarnings("ignore") 

from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Fri Jun 10 20:18:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 33%   42C    P8    21W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 36%   

# MongoDB

In [4]:
from pymongo import MongoClient
import pymongo

In [5]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn

    db_rm_cn_2012 = db.rm_patent_cn_2012
    db_rm_us_2012 = db.rm_patent_us_2012
    db_rm_de_2012 = db.rm_patent_de_2012

    
except errors.ConnectionFailure as err:
    print(err)

# 2012

## Company數量

In [6]:
data_assignee = db_rm_de_2012.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [7]:
df_family_de = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [8]:
data_assignee = db_rm_cn_2012.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_cn = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [9]:
data_assignee = db_rm_us_2012.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_us = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [10]:
final_df = pd.concat([df_family_de,df_family_cn,df_family_us],axis=0)

In [11]:
# 取總數前30家公司
final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)

,company,count
213,Jiangsu IoT Research and Development Center,100
271,Nanjing IoT Sensor Technology Co Ltd,52
87,Cisco Technology Inc,33
83,China core Microelectronics Technology Chengdu...,25
260,NANJING AQUAPEL IOT CO Ltd,20
26,"BEIJING GOLDENET IOT TECHNOLOGY CO., LTD.",18
466,Xerox Corp,17
137,"Gaoxing Zhilian Technology Co., Ltd.",17
187,Institute of Microelectronics of CAS,13
208,Jiangsu CAS IGBT Technology Co Ltd,12


In [12]:
com_df = final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)
com_df.reset_index(drop=True, inplace=True)

In [13]:
com_df["count_log"] = com_df["count"].apply(lambda x: np.log(x))
com_df

,company,count,count_log
0,Jiangsu IoT Research and Development Center,100,4.605170
1,Nanjing IoT Sensor Technology Co Ltd,52,3.951244
2,Cisco Technology Inc,33,3.496508
3,China core Microelectronics Technology Chengdu...,25,3.218876
4,NANJING AQUAPEL IOT CO Ltd,20,2.995732
5,"BEIJING GOLDENET IOT TECHNOLOGY CO., LTD.",18,2.890372
6,Xerox Corp,17,2.833213
7,"Gaoxing Zhilian Technology Co., Ltd.",17,2.833213
8,Institute of Microelectronics of CAS,13,2.564949
9,Jiangsu CAS IGBT Technology Co Ltd,12,2.484907


公司名稱正規化

In [14]:
com_df["company"] = com_df.company.apply(lambda x: x.replace(" Inc","")\
                                                    .replace(" Co Ltd","")\
                                                    .replace(" Co. Ltd.","")\
                                                    .replace(" Co., Ltd.","")\
                                                    .replace(" Ltd","")\
                                                    .replace(" Corp","")\
                                                    .replace("..","")\
                                                    .replace("International Business Machines","IBM")\
                                                    .replace("Nippon Telegraph and Telephone","Nippon Telegraph & Tel")\
                                                    .replace("Alibaba Group Holding Ltd","Alibaba")\
                                                    .replace("ZTE Intelligent IoT Technology","ZTE"))

In [15]:
com_df

,company,count,count_log
0,Jiangsu IoT Research and Development Center,100,4.605170
1,Nanjing IoT Sensor Technology,52,3.951244
2,Cisco Technology,33,3.496508
3,China core Microelectronics Technology Chengdu...,25,3.218876
4,NANJING AQUAPEL IOT CO,20,2.995732
5,"BEIJING GOLDENET IOT TECHNOLOGY CO., LTD.",18,2.890372
6,Xerox,17,2.833213
7,Gaoxing Zhilian Technology,17,2.833213
8,Institute of Microelectronics of CAS,13,2.564949
9,Jiangsu CAS IGBT Technology,12,2.484907


In [16]:
com_ls = list(com_df.company)

# Data 2012

富比士2000資料

In [17]:
f_data = pd.read_csv("./Forbes/Forbes_2012.csv")

In [18]:
f_data[f_data["Industry"]=="Semiconductors"]

,Company,Industry,Country,Market Value,Sales,Profits,Assets,Rank,Forbes Webpage,Profits as % of Assets,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
25,Samsung Electronics,Semiconductors,South Korea,162.0,142.4,11.5,133.7,26,http://www.forbes.com/companies/samsung-electr...,0.086013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,Intel,Semiconductors,United States,138.5,54.0,12.9,71.1,85,http://www.forbes.com/companies/intel/,0.181435,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
283,Taiwan Semiconductor,Semiconductors,Taiwan,71.1,14.1,4.4,25.3,284,http://www.forbes.com/companies/taiwan-semicon...,0.173913,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,Texas Instruments,Semiconductors,United States,37.8,13.7,2.2,20.5,360,http://www.forbes.com/companies/texas-instrume...,0.107317,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
583,Applied Materials,Semiconductors,United States,16.6,10.0,1.5,13.6,583,http://www.forbes.com/companies/applied-materi...,0.110294,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689,ASML Holding,Semiconductors,Netherlands,20.0,7.3,1.9,9.4,690,http://www.forbes.com/companies/asml-holding/,0.202128,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
782,Broadcom,Semiconductors,United States,20.6,7.4,0.9,9.0,783,http://www.forbes.com/companies/broadcom/,0.100000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
867,Tokyo Electron,Semiconductors,Japan,10.3,8.0,0.9,9.5,868,http://www.forbes.com/companies/tokyo-electron/,0.094737,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
948,Hynix Semiconductor,Semiconductors,South Korea,17.8,9.4,-0.1,14.9,948,http://www.forbes.com/companies/hynix-semicond...,-0.006711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
953,STMicroelectronics,Semiconductors,Switzerland,7.5,9.0,0.6,10.9,954,http://www.forbes.com/companies/stmicroelectro...,0.055046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
Ind_ls = list(f_data[f_data["Industry"]=="Semiconductors"].Company)

## 使用company top 30 做join

In [20]:
forb_ls = list(np.where(com_df.company.isin(Ind_ls), 1, 0))

In [21]:
com_df[com_df.company.isin(Ind_ls)]

,company,count,count_log
17,Samsung Electronics,8,2.079442


In [22]:
# pbc 0.1, 0.1
comp_ls = ["China Petroleum and Chemical","Gaoxing Zhilian Technology","Huawei Technologies","Institute of Microelectronics of CAS","Institute of Microelectronics of CAS","Jiangsu CAS IGBT Technology","Jiangsu IoT Research and Development Center","Kingfa Science and Technology","Nanjing IoT Sensor Technology","Samsung Electronics","Seiko Epson","ZTE"]

In [23]:
competitor_ls = list(np.where(com_df.company.isin(comp_ls), 1, 0))

In [24]:
com_df[com_df.company.isin(comp_ls)]

,company,count,count_log
0,Jiangsu IoT Research and Development Center,100,4.605170
1,Nanjing IoT Sensor Technology,52,3.951244
7,Gaoxing Zhilian Technology,17,2.833213
8,Institute of Microelectronics of CAS,13,2.564949
9,Jiangsu CAS IGBT Technology,12,2.484907
11,ZTE,10,2.302585
12,Seiko Epson,10,2.302585
13,Huawei Technologies,9,2.197225
15,Kingfa Science and Technology,9,2.197225
16,China Petroleum and Chemical,9,2.197225


In [25]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.08575808820545287
ari:  0.048223350253807105


## 使用Forbes做join

In [26]:
forb_ls = list(np.where(f_data.Company.isin(Ind_ls), 1, 0))

In [27]:
# pbc 0.1, 0.1
comp_ls = ["China Petroleum and Chemical","Gaoxing Zhilian Technology","Huawei Technologies","Institute of Microelectronics of CAS","Institute of Microelectronics of CAS","Jiangsu CAS IGBT Technology","Jiangsu IoT Research and Development Center","Kingfa Science and Technology","Nanjing IoT Sensor Technology","Samsung Electronics","Seiko Epson","ZTE"]

In [28]:
competitor_ls = list(np.where(f_data.Company.isin(comp_ls), 1, 0))

In [29]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.027221746940671476
ari:  0.05902618385424741
